In [1]:
import re
import os
import pickle
import subprocess
import youtube_dl
import multiprocessing
import tqdm

In [7]:
DATA_DIR = '../../data'
DATA_NAME = DATA_DIR + '/10vid'
DATA_AUDIO = DATA_DIR + '/10vid-audio'
#DATA_NAME = DATA_DIR + '/YouTubeClips'
#DATA_AUDIO = DATA_DIR + '/YouTubeClips-audio'


### Extract audio from video

In [8]:
def check_audio(file_path):
    '''Check whether there is audio in the video or not, True if there is audio'''
    
    cmd =("ffprobe -i {0} -show_streams -select_streams a  -loglevel error".format(file_path))
    output = subprocess.check_output(cmd,shell=True)
    return (output != b'')

In [10]:
# load video list from video folder
video_ids = os.listdir(DATA_NAME)

# extract feature for each video which is a batch of frames
print("Starting to process...")
count = 1
audio_count = 0
n = len(video_ids)
for video_id in video_ids:
    video_path = DATA_NAME+'/'+video_id
    if os.path.isfile(video_path):
        count = count+1
        if check_audio(video_path):
            audio_path = DATA_AUDIO+'/'+video_id[:-4]+'.wav'
            audio_count = audio_count + 1
        if (count % 100 == 0):
            print("processing: {0}/{1}".format(count,n))
            
#             cmd = ("ffmpeg -i {0} -ab 160k -ac 2 -ar 44100 -vn {1}".format(video_path, audio_path))
#             subprocess.call(cmd, shell=True)

print("Finished")
print("audio files: " + str(audio_count))


Starting to process...
processing: 100/1970
processing: 200/1970
processing: 300/1970
processing: 400/1970
processing: 500/1970
processing: 600/1970
processing: 700/1970
processing: 800/1970
processing: 900/1970
processing: 1000/1970
processing: 1100/1970
processing: 1200/1970
processing: 1300/1970
processing: 1400/1970
processing: 1500/1970
processing: 1600/1970
processing: 1700/1970
processing: 1800/1970
processing: 1900/1970
Finished
audio files: 0


In [7]:
video_id = '_UqnTFs1BLc_23_27.avi'
video_path = DATA_NAME+'/'+video_id
audio_path = DATA_AUDIO+'/'+video_id[:-4]+'.wav'
cmd = ("ffmpeg -i {0} -ab 160k -ac 2 -ar 44100 -vn {1}".format(video_path, audio_path))
subprocess.call(cmd, shell=True)

1

### Download audio from youtube

In [9]:
def get_audio(video_file, audio_path=DATA_AUDIO):
    # get video info
    video_id = re.sub('_\d*_\d*\.avi$', '', video_file)
    audio_range = re.search('_(\d*)_(\d*)\.avi$', video_file)
    start = audio_range.group(1)
    duration = int(audio_range.group(2)) - int(audio_range.group(1)) + 1
    
    # set output path
    OUTPUT_TEMP = '"' + audio_path + '/%(id)s_' + audio_range.group(1) + '_' + audio_range.group(2) + '.%(ext)s' + '"'
    # construct command
    cmd =('youtube-dl -o {0} -x -f m4a --no-continue --postprocessor-args "-ss {1} -t {2}" http://youtube.com/watch?v={3}'.format(OUTPUT_TEMP, start, duration, video_id))
    
    return subprocess.call(cmd,shell=True)

In [12]:
def process_file(video_file):
    video_path = DATA_NAME+'/'+video_file
    video_id = re.sub('_\d*_\d*\.avi$', '', video_file)
    audio_path = DATA_AUDIO + '/' + video_file[:-4] + '.m4a'
    if not os.path.isfile(audio_path):
        get_audio(video_file)

In [13]:
# load video list from video folder
video_files = os.listdir(DATA_NAME)

pool = multiprocessing.Pool(multiprocessing.cpu_count())
for _ in tqdm.tqdm(pool.imap_unordered(process_file, video_files), total=len(video_files)):
    pass


100%|██████████| 11/11 [01:02<00:00,  5.84s/it]


In [84]:
# load video list from video folder
video_files = os.listdir(DATA_NAME)

print("Starting to process...")
count = 0
audio_count = 0
n = len(video_files)
for video_file in video_files:
    video_path = DATA_NAME+'/'+video_file
    video_id = re.sub('_\d*_\d*\.avi$', '', video_file)
    audio_path = DATA_AUDIO + '/' + video_id + '.m4a'
    count = count+1
    if not os.path.isfile(audio_path):
        get_audio(DATA_AUDIO, video_file)
        if (count % 2 == 0):
            print("processing: {0}/{1}".format(count,n))

print("Finished")


Starting to process...
processing: 2/10
processing: 4/10
processing: 6/10
processing: 8/10
processing: 10/10
Finished


In [79]:
get_audio(DATA_AUDIO, '_UqnTFs1BLc', 10,4)

0

In [59]:
video_file = '_UqnTFs1BLc_23_27.avi'
audio_range = re.search('_(\d*)_(\d*)\.avi$', video_file)
audio_range.group(1)

'23'

In [30]:
video_id = '_UqnTFs1BLc'
OUTPUT_TEMP = '"' + DATA_AUDIO + '/%(id)s.%(ext)s' + '"'
cmd =('youtube-dl -o {0} -x --no-continue --postprocessor-args "-ss 00:00:23.00 -t 00:00:04.00" http://youtube.com/watch?v={1}'.format(OUTPUT_TEMP, video_id))
subprocess.call(cmd,shell=True)

0

In [21]:
OUTPUT_TEMP = '"' + DATA_AUDIO + '/%(id)s.%(ext)s' + '"'
OUTPUT_TEMP

'"../../data/10vid-audio/%(id)s.%(ext)s"'

In [24]:
('youtube-dl --id -o {0} -x --postprocessor-args "-ss 00:00:23.00 -t 00:00:04.00" http://youtube.com/watch?v={1}'.format(OUTPUT_TEMP, video_id))

'youtube-dl --id -o "../../data/10vid-audio/%(id)s.%(ext)s" -x --postprocessor-args "-ss 00:00:23.00 -t 00:00:04.00" http://youtube.com/watch?v=_UqnTFs1BLc'